<a href="https://colab.research.google.com/github/parishtherealone/github_demo/blob/main/ImprovisedImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = y_train.reshape(-1,)
y_test = y_test.reshape(-1,)


In [ ]:
BATCH_SIZE = 64
IMG_SIZE = 96   # you can reduce to 64 if RAM crashes again

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)




In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False


In [ ]:
model = models.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(10, activation='softmax')
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=5,
    restore_best_weights=True
)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=40,
    callbacks=[early_stop]
)




Epoch 1/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 34ms/step - accuracy: 0.3070 - loss: 2.2284 - val_accuracy: 0.6635 - val_loss: 0.9888
Epoch 2/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.5328 - loss: 1.3784 - val_accuracy: 0.7093 - val_loss: 0.8476
Epoch 3/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 33ms/step - accuracy: 0.5785 - loss: 1.2182 - val_accuracy: 0.7321 - val_loss: 0.7847
Epoch 4/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 33ms/step - accuracy: 0.6066 - loss: 1.1438 - val_accuracy: 0.7434 - val_loss: 0.7504
Epoch 5/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.6178 - loss: 1.0985 - val_accuracy: 0.7521 - val_loss: 0.7274
Epoch 6/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 34ms/step - accuracy: 0.6372 - loss: 1.0461 - val_accuracy: 0.7596 - val_loss: 0.7057
Epoch 7/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.6478 - loss: 1.0210 - val_accuracy: 0.7582 - val_loss: 0.7042
Epoch 8/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.6546 - loss: 0.9967 - 

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history_fine = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=[early_stop]
)


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 52ms/step - accuracy: 0.5800 - loss: 1.2784 - val_accuracy: 0.7610 - val_loss: 0.7795
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 49ms/step - accuracy: 0.6534 - loss: 1.0317 - val_accuracy: 0.7826 - val_loss: 0.6700
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.6806 - loss: 0.9432 - val_accuracy: 0.7909 - val_loss: 0.6314
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 51ms/step - accuracy: 0.6942 - loss: 0.8846 - val_accuracy: 0.7976 - val_loss: 0.6049
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.7122 - loss: 0.8396 - val_accuracy: 0.8013 - val_loss: 0.5924
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.7249 - loss: 0.8009 - val_accuracy: 0.8095 - val_loss: 0.5690
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.7310 - loss: 0.7837 - val_accuracy: 0.8145 - val_loss: 0.5575
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.7408 - loss: 0.7521 - 

In [ ]:
model.evaluate(test_ds)


157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.8451 - loss: 0.4668


[0.47457510232925415, 0.8407999873161316]